In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import openpyxl
import xlrd
from rdkit import Chem
from rdkit.Chem import inchi
import papyrus_scripts
from chembl_structure_pipeline import standardizer as ChEMBL_standardizer
from papyrus_structure_pipeline import standardizer as Papyrus_standardizer
from papyrus_structure_pipeline import standardize

[<class 'str'>, typing.Iterable[str]]
[typing.Tuple[int, int], typing.Tuple[int]]
[<class 'pyparsing.results.ParseResults'>, typing.Sequence[pyparsing.results.ParseResults]]
[typing.Callable[[], typing.Any], typing.Callable[[pyparsing.results.ParseResults], typing.Any], typing.Callable[[int, pyparsing.results.ParseResults], typing.Any], typing.Callable[[str, int, pyparsing.results.ParseResults], typing.Any]]
[typing.Callable[[], bool], typing.Callable[[pyparsing.results.ParseResults], bool], typing.Callable[[int, pyparsing.results.ParseResults], bool], typing.Callable[[str, int, pyparsing.results.ParseResults], bool]]
[typing.Callable[[str, int, ForwardRef('ParserElement'), bool], NoneType], <class 'NoneType'>]
[typing.Callable[[str, int, int, ForwardRef('ParserElement'), pyparsing.results.ParseResults, bool], NoneType], <class 'NoneType'>]
[typing.Callable[[str, int, ForwardRef('ParserElement'), Exception, bool], NoneType], <class 'NoneType'>]
[<class 'pyparsing.results.ParseResults'>

[11:59:39] Initializing Normalizer


In [2]:

from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm

Query data from Papyrus

In [3]:
from papyrus_scripts import download_papyrus
download_papyrus(only_pp=False)

Latest version: 05.6
Number of files to be downloaded: 10
Total size: 8.18GB


In [1]:
from papyrus_scripts import read_papyrus
data = read_papyrus(plusplus=False,chunksize=1_000_000)

[<class 'bytes'>, typing.List[bytes]]
[typing.List[charset_normalizer.models.CharsetMatch], <class 'NoneType'>]
[ForwardRef('CharsetMatch'), <class 'NoneType'>]
[<class 'bytes'>, <class 'bytearray'>]
[typing.List[str], <class 'NoneType'>]
[<class 'str'>, <class 'bytes'>, <class 'os.PathLike'>]
[<class 'os.PathLike'>, <class 'str'>, <class 'typing.BinaryIO'>, <class 'bytes'>]
[<class 'str'>, <class 'float'>]
[<class 'str'>, <class 'float'>, <class 'NoneType'>]
[<class 'bytes'>, <class 'str'>]
[<class 'str'>, <class 'bytes'>, <class 'bytearray'>]
[typing.Mapping[str, str], <class 'NoneType'>]
[<class 'str'>, <class 'pathlib.Path'>]
[typing.Mapping[str, typing.Any], <class 'NoneType'>]
[<class 'str'>, typing.Sequence[str]]
[ForwardRef('botocore.client.BaseClient'), <class 'NoneType'>]
[~X, <class 'NoneType'>]
[typing.Type[~X], <class 'NoneType'>]
[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>]
[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>, ForwardRef('Index'), ForwardRef(

2024-02-14 11:45:31,796	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


[<class 'str'>, <class 'logging.Logger'>]
[ForwardRef('RepeatedScalarFieldContainer[_T]'), typing.Iterable[~_T]]
[<class 'int'>, <class 'NoneType'>]
[<class 'int'>, <class 'slice'>]
[ForwardRef('RepeatedCompositeFieldContainer[_T]'), typing.Iterable[~_T]]
[~_V, <class 'NoneType'>]
[~_V, ~_T]
[<class 'dict'>, <class 'NoneType'>]
[<class 'bool'>, <class 'NoneType'>]
[typing.IO[~AnyStr], <class 'NoneType'>]
[typing.List[int], <class 'NoneType'>]
[typing.Callable[[], NoneType], <class 'NoneType'>]
[ForwardRef('_BarManager'), <class 'NoneType'>]
[typing.Iterable, <class 'NoneType'>]
[typing.List[str], <class 'NoneType'>]
[typing.Dict[str, typing.Any], ForwardRef('RuntimeEnv')]
[typing.Dict[str, typing.Any], ForwardRef('RuntimeEnv'), <class 'NoneType'>]
[ForwardRef('grpc.ChannelCredentials'), <class 'NoneType'>]
[typing.Dict[str, typing.Any], <class 'NoneType'>]
[<class 'float'>, <class 'NoneType'>]
[typing.Sequence[typing.Tuple[str, typing.Any]], <class 'NoneType'>]
[<class 'str'>, <class '

In [3]:
influx_ac_list = ['O76082','Q96FL8','P46721','O94956','Q9NYB5','Q6ZNC8','Q6ZWT7','Q95T53','Q01650']

In [4]:
from papyrus_scripts import read_protein_set
protein_data = read_protein_set()

In [5]:
from papyrus_scripts import keep_quality
from papyrus_scripts import keep_accession
filter1=keep_quality(data,'High')
filter2=keep_accession(filter1, influx_ac_list)

In [6]:
from papyrus_scripts import consume_chunks
influx = consume_chunks(filter2)

0it [00:00, ?it/s]

In [8]:
len(influx)

74

In [13]:
#Check for molecules that are present both in training and validation dataset: remove from validation

train = pd.read_csv('train_data/kadar_influx_train.csv', delimiter=',', index_col=0)

In [15]:
mask = influx['connectivity'].isin(train['inchi_connectivity'])
common_values_list = influx[mask]['connectivity'].tolist()
print(f'Common: {len(common_values_list)}')

influx_val = influx[~mask]
print(f'Final: {len(influx_val)}')

56

In [17]:
#Keep the instance with the highest pchembl

val_influx = influx_val.loc[influx_val.groupby('InChIKey')['pchembl_value_Mean'].idxmax()]

In [18]:
#Assign classes

def compare_values(row):
    if row['pchembl_value_Mean'] <= 5:
        return 'Non-substrate'
    elif row['pchembl_value_Mean'] > 5:
        return 'Substrate'
    else:
        return '??'

val_influx['Class'] = val_influx.apply(compare_values, axis=1)

,Activity_ID,Quality,source,CID,SMILES,connectivity,InChIKey,InChI,InChI_AuxInfo,target_id,...,Activity_class,relation,pchembl_value,pchembl_value_Mean,pchembl_value_StdDev,pchembl_value_SEM,pchembl_value_N,pchembl_value_Median,pchembl_value_MAD,Class
112670,AXRYRYVKAWYZBR_on_O94956_WT,High,ChEMBL31,ChEMBL31.compound.42310,COC(=O)NC(C(=O)NC(Cc1ccccc1)C(O)CN(Cc1ccc(-c2n...,AXRYRYVKAWYZBR,AXRYRYVKAWYZBR-UHFFFAOYSA-N,"InChI=1S/C38H52N6O7/c1-37(2,3)31(41-35(48)50-7...","""AuxInfo=1/1/N:49,50,51,45,46,47,1,43,15,14,16...",O94956_WT,...,NaN,=,5.290; 5.280,5.285,0.005,0.005,2.0,5.285,0.005,Substrate
113347,AXUZQJFHDNNPFG_on_O94956_WT,High,ChEMBL31,ChEMBL31.compound.3406,CN(C)C(=O)CCSC(SCCC(=O)O)c1cccc(C=Cc2nc3c(ccc(...,AXUZQJFHDNNPFG,AXUZQJFHDNNPFG-UHFFFAOYSA-N,InChI=1S/C26H27ClN2O3S2/c1-29(2)24(30)12-14-33...,"""AuxInfo=1/1/N:1,3,18,19,17,21,27,32,28,22,33,...",O94956_WT,...,NaN,=,5.540; 5.550,5.545,0.005,0.005,2.0,5.545,0.005,Substrate
184167,BMLIIPOXVWESJG_on_O94956_WT,High,ChEMBL31,ChEMBL31.compound.262785,COc1c(O)ccc(C2Oc3c(cc(C4Oc5c(C(=O)C4O)c(O)cc(O...,BMLIIPOXVWESJG,BMLIIPOXVWESJG-UHFFFAOYSA-N,InChI=1S/C25H22O10/c1-33-18-6-10(2-3-15(18)28)...,"""AuxInfo=1/0/N:1,7,6,13,29,35,25,28,33,8,14,26...",O94956_WT,...,NaN,=,5.440,5.440,0.000,0.000,1.0,5.440,0.000,Substrate
417712,DJSISFGPUUYILV_on_O94956_WT,High,ChEMBL31,ChEMBL31.compound.107019,O=C(O)C1OC(Oc2c(O)c(O)c3c(=O)cc(-c4ccc(O)cc4)o...,DJSISFGPUUYILV,DJSISFGPUUYILV-UHFFFAOYSA-N,InChI=1S/C21H18O12/c22-8-3-1-7(2-4-8)10-5-9(23...,"""AuxInfo=1/1/N:19,24,20,23,16,27,18,21,14,17,2...",O94956_WT,...,NaN,=,4.590,4.590,0.000,0.000,1.0,4.590,0.000,Non-substrate
701810,GSDSWSVVBLHKDQ_on_Q96FL8_WT,High,ChEMBL31,ChEMBL31.compound.148494,CC1COc2c3c(cc(F)c2N2CCN(C)CC2)c(=O)c(C(=O)O)cn31,GSDSWSVVBLHKDQ,GSDSWSVVBLHKDQ-UHFFFAOYSA-N,InChI=1S/C18H20FN3O4/c1-10-9-26-17-14-11(16(23...,"""AuxInfo=1/1/N:1,16,14,17,13,18,8,25,3,2,7,21,...",Q96FL8_WT,...,NaN,=,4.750,4.750,0.000,0.000,1.0,4.750,0.000,Non-substrate


In [30]:
#Keep only the relevant columns, code classes, and rename column

val_influx = val_influx[['SMILES','connectivity','InChIKey','pchembl_value_Mean','Class']]
val_influx['Class'] = val_influx['Class'].replace({'Substrate':1, 'Non-substrate':0})
val_influx.rename(columns={'Class': 'status_influx', 'connectivity': 'inchi_connectivity','SMILES':'papyrus_SMILES'}, inplace=True)

KeyError: "['SMILES', 'connectivity', 'Class'] not in index"

In [33]:
#Reset index
val_influx = val_influx.reset_index(drop=True)

In [35]:
#Save the influx molecules
val_influx.to_csv('val_data/kadar_val_influx.csv', index=True)